# Where is your GAN? Генеративно-состязательная нейросеть: ваша первая модель на PyTorch

Данный материал представляет собой незначительно сокращенный перевод публикации Ренато Кандидо [Generative Adversarial Networks: Build Your First Models](https://realpython.com/generative-adversarial-networks/).

Генеративно-состязательные сети (англ. Generative adversarial networks, сокр. GAN) – [нейронные сети](https://ru.wikipedia.org/wiki/%D0%9D%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D0%B0%D1%8F_%D1%81%D0%B5%D1%82%D1%8C), которые умеют генерировать изображения, музыку, речь и тексты, похожие на те, что делают люди. 

GAN были активной темой исследований последних лет. Директор по исследованиям в области искусственного интеллекта в Facebook Ян Лекан назвал состязательное обучение «самой интересной идеей в области машинного обучения за последние 10 лет». Ниже вы узнаете, как работают GAN и создадите две собственные модели. Для работы с моделями будет использоваться [фреймворк глубокого обучения](https://proglib.io/p/dl-frameworks) PyTorch.

# Что такое генеративно-состязательная нейросеть?

[Генеративно-состязательные сети](https://ru.wikipedia.org/wiki/%D0%93%D0%B5%D0%BD%D0%B5%D1%80%D0%B0%D1%82%D0%B8%D0%B2%D0%BD%D0%BE-%D1%81%D0%BE%D1%81%D1%82%D1%8F%D0%B7%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%B0%D1%8F_%D1%81%D0%B5%D1%82%D1%8C) – это модели машинного обучения, умеющие имитировать заданное распределение данных. Впервые они были предложены в [статье NeurIPS](https://papers.nips.cc/paper/5423-generative-adversarial-nets.pdf) 2014 г. экспертом в глубоком обучении Яном Гудфеллоу и его коллегами.

GAN состоят из двух нейронных сетей, одна из которых обучена генерировать данные, а другая обучена отличать ложные данные от реальных (отсюда и «состязательный» характер модели). GAN показывают впечатляющие результаты в отношении генерации изображений и видео, такие как:
* Перенос стилей ([CycleGAN](https://github.com/junyanz/CycleGAN/)) – преобразование одного изображения в соответствии со стилем других изображений (например, картин известного художника)
* Генерация человеческих лиц ([StyleGAN](https://en.wikipedia.org/wiki/StyleGAN)), реалистичные примеры которых вы можете найти на сайте [This Person Does Not Exist](https://www.thispersondoesnotexist.com/).

GAN и другие структуры, генерирующие данные, называют **генеративными моделями** в противовес более широко изученным **дискриминативным моделям**. Прежде чем погрузиться в GAN, посмотрим на различия между этими двумя типами моделей.

# Сравнение дискриминативных и генеративных моделей

Дискриминативные модели используются для большинства [обучения с учителем](https://ru.wikipedia.org/wiki/%D0%9E%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5_%D1%81_%D1%83%D1%87%D0%B8%D1%82%D0%B5%D0%BB%D0%B5%D0%BC) на [классификацию](https://ru.wikipedia.org/wiki/%D0%97%D0%B0%D0%B4%D0%B0%D1%87%D0%B0_%D0%BA%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%86%D0%B8%D0%B8) или [регрессию](https://ru.wikipedia.org/wiki/%D0%A0%D0%B5%D0%B3%D1%80%D0%B5%D1%81%D1%81%D0%B8%D0%BE%D0%BD%D0%BD%D1%8B%D0%B9_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7). В качестве примера проблемы классификации предположим, что вы хотите обучить [модель классификации изображений рукописных цифр от 0 до 9](https://proglib.io/p/neural-network-course). Для этого вы можете использовать маркированный набор данных, содержащий изображения рукописных цифр и связанные метки, указывающие соответствие цифр и изображений.

В процессе обучения для настройки параметров модели вы будете использовать специальный алгоритм. Его цель состоит в том, чтобы [минимизировать функцию потерь](https://ru.wikipedia.org/wiki/%D0%A4%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D1%8F_%D0%BF%D0%BE%D1%82%D0%B5%D1%80%D1%8C) – критерий раскхождения между истинным значением оцениваемого параметра и его оценкой. После фазы обучения вы можете использовать модель для классификации нового изображения рукописной цифры, сопоставив входному изображению наиболее вероятную цифру, как показано на рисунке ниже.

![](https://files.realpython.com/media/fig_discriminative.9c22a1cd877d.png)
*Схема обучения дискриминативной модели*

Дискриминативную модель для задач классификации можно представить, как «черный ящик», который использует обучающие данные для изучения границ между классами. Найденные границы далее используются моделью, чтобы различить входные данные – предсказать их класс. В математическом отношении дискриминативные модели изучают [условную вероятность](https://ru.wikipedia.org/wiki/%D0%A3%D1%81%D0%BB%D0%BE%D0%B2%D0%BD%D0%B0%D1%8F_%D0%B2%D0%B5%D1%80%D0%BE%D1%8F%D1%82%D0%BD%D0%BE%D1%81%D1%82%D1%8C) $P(y|x)$ наблюдения $y$ при заданном входе $x$.

Дискриминативные модели это не обязательно нейронные сети. К ним также относятся такие модели машинного обучения, как [логистическая регрессия](https://ru.wikipedia.org/wiki/%D0%9B%D0%BE%D0%B3%D0%B8%D1%81%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B5%D0%B3%D1%80%D0%B5%D1%81%D1%81%D0%B8%D1%8F) и [метод опорных векторов (SVM)](https://ru.wikipedia.org/wiki/%D0%9C%D0%B5%D1%82%D0%BE%D0%B4_%D0%BE%D0%BF%D0%BE%D1%80%D0%BD%D1%8B%D1%85_%D0%B2%D0%B5%D0%BA%D1%82%D0%BE%D1%80%D0%BE%D0%B2).

В то время как дискриминативные модели используются для контролируемого обучения, генеративные модели часто используют неразмеченный набор данных, то есть могут рассматриваться как форма [обучения без учителя](https://ru.wikipedia.org/wiki/%D0%9E%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B1%D0%B5%D0%B7_%D1%83%D1%87%D0%B8%D1%82%D0%B5%D0%BB%D1%8F). Используя набор данных из рукописных цифр, вы можете обучить генеративную модель для генерации новых цифр. На этапе обучения модель использует определенный алгоритм для настройки параметров модели, чтобы также минимизировать функцию потерь и определить распределение вероятностей обучающего набора.

![](https://files.realpython.com/media/fig_generative.5f01c08f5208.png)
*Схема обучения генеративной модели*

В отличие от дискриминативных моделей, генеративные модели изучают свойства [функции вероятности](https://ru.wikipedia.org/wiki/%D0%A4%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D1%8F_%D0%B2%D0%B5%D1%80%D0%BE%D1%8F%D1%82%D0%BD%D0%BE%D1%81%D1%82%D0%B8) $P(x)$ входных данных $x$. В результате они порождают не предсказание, а новый объект со свойствами, родственными обучающему набору данных.

---

**Примечание**. Генеративные модели также можно использовать и для размеченных наборов данных. Их также можно использовать для задач классификации, но в целом дискриминативные модели работают лучше, когда речь идет о классификации. 

Вы можете найти больше информации об относительных сильных и слабых сторонах  классификаторов в статье [«Дискриминативные и генеративные классификаторы: сравнение логистической регрессии и наивного байесовского алгоритма»](https://realpython.com/generative-adversarial-networks/) (англ.).

---

Помимо GAN существуют другие генеративные модели архитектуры:
* [Машина Больцмана](https://ru.wikipedia.org/wiki/%D0%9C%D0%B0%D1%88%D0%B8%D0%BD%D0%B0_%D0%91%D0%BE%D0%BB%D1%8C%D1%86%D0%BC%D0%B0%D0%BD%D0%B0)
* [Автокодировщик](https://ru.wikipedia.org/wiki/%D0%90%D0%B2%D1%82%D0%BE%D0%BA%D0%BE%D0%B4%D0%B8%D1%80%D0%BE%D0%B2%D1%89%D0%B8%D0%BA)
* [Скрытая марковская модель](https://ru.wikipedia.org/wiki/%D0%A1%D0%BA%D1%80%D1%8B%D1%82%D0%B0%D1%8F_%D0%BC%D0%B0%D1%80%D0%BA%D0%BE%D0%B2%D1%81%D0%BA%D0%B0%D1%8F_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D1%8C)
* Модели, предсказывающие следующее слово в последовательности, например, [GPT-2](https://en.wikipedia.org/wiki/OpenAI#GPT-2)

Тем не менее, в последнее время GAN привлекли наибольшее внимание общественности благодаря впечатляющим результатам в генерации изображений и видео. Поэтому остановимся на ее устройстве подробнее.

# Архитектура генеративно-состязательных нейросетей

Генеративно-состязательная сеть это на самом деле не одна сеть, а две: генератор и дискриминатор. Роль **генератора** состоит в том, чтобы на основе реальной выборки сгенерировать набор данных, напоминающий реальные данные. **Дискриминатор** в свою очередь обучен оценивать вероятность того, что данный образец получен из реальных данных, а не предоставлен генератором. Состязательность GAN заключается в том, что генератор и дискриминатор играют в кошки-мышки: генератор пытается обмануть дискриминатор, а дискриминатор старается лучше идентифицировать сгенерированные выборки.

Чтобы понять, как работает обучение GAN, рассмотрим игрушечный пример с набором данных, состоящим из двумерных выборок $(x_1, x_2)$, с $x_1$ в интервале от $0$ до $2π$ и $x_2 = sin(x_1)$, как показано на следующем рисунке.

![](https://files.realpython.com/media/fig_x1x2.f8a39d8ff58a.png)

Общая структура GAN для генерации пар $(x̃_1, x̃_2)$, напоминающих точки набора данных, показана на следующем рисунке.

![](https://files.realpython.com/media/fig_gan.4f0f744c7999.png)

Генератор $G$ получает на вход пары случайных чисел ($z_1, z_2$), преобразуя их так, чтобы они напоминали реальные выборки. Структура нейронной сети $G$ может быть произвольной, например, [многослойный персептрон](https://ru.wikipedia.org/wiki/%D0%9C%D0%BD%D0%BE%D0%B3%D0%BE%D1%81%D0%BB%D0%BE%D0%B9%D0%BD%D1%8B%D0%B9_%D0%BF%D0%B5%D1%80%D1%86%D0%B5%D0%BF%D1%82%D1%80%D0%BE%D0%BD_%D0%A0%D1%83%D0%BC%D0%B5%D0%BB%D1%8C%D1%85%D0%B0%D1%80%D1%82%D0%B0) (MLP) или [сверточная нейронная сеть](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BE%D1%87%D0%BD%D0%B0%D1%8F_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D0%B0%D1%8F_%D1%81%D0%B5%D1%82%D1%8C) (CNN).

На вход дискриминатора $D$ попеременно поступают реальные образцы из обучающего набора данных и смоделированные образцы, предоставленные генератором $G$. Роль дискриминатора заключается в оценке вероятности того, что входные данные принадлежат реальному набору данных. То есть обучение выполняется таким образом, чтобы $D$ выдавал $1$, когда получает реальный образец, и $0$, когда получает сгенерированный образец.

Как и в случае с генератором, вы можете выбрать любую структуру нейронной сети для $D$ с учетом размеров входных и выходных данных. В рассматриваемом примере ввод является двумерным, а выходные данные – [скаляром](https://ru.wikipedia.org/wiki/%D0%A1%D0%BA%D0%B0%D0%BB%D1%8F%D1%80%D0%BD%D0%B0%D1%8F_%D0%B2%D0%B5%D0%BB%D0%B8%D1%87%D0%B8%D0%BD%D0%B0) в диапазоне от 0 до 1.

Процесс обучения GAN заключается в [минимаксной игре](https://ru.wikipedia.org/wiki/%D0%9C%D0%B8%D0%BD%D0%B8%D0%BC%D0%B0%D0%BA%D1%81) двух игроков, в которой $D$ адаптирован для минимизации ошибки различия реального и сгенерированного образца, а $G$ адаптирован на максимизацию вероятности того, что $D$ допустит ошибку.

На каждом этапе обучения происходит обновление параметров моделей $D$ и $G$. Чтобы обучить $D$, на каждой итерации мы помечаем некоторую выбору реальных образцов из обучающих данных, как 1, а выборку сгенерированных образцов, созданных $G$, как 0. Таким образом, для обновления параметров $D$, как показано на следующей схеме, можно использовать обычную схему обучения с учителем.

![](https://files.realpython.com/media/fig_train_discriminator.cd1a1e32764f.png)
*Процесс обучения дискриминатора*

Для каждой партии обучающих данных, содержащих помеченные реальные и сгенерированные образцы, мы обновляем набор параметров модели $D$, минимизируя тем самым функцию потерь. После того, как параметры $D$ обновлены, мы обучаем $G$ генерировать более качественные образцы. Набор параметров D «замораживается» на время обучения генератора.

![](https://files.realpython.com/media/fig_train_generator.7196c4f382ba.png)

Когда $G$ будет генерировать образцы настолько хорошо, что $D$ начнет обманываться, выходная вероятность устремится к 1 – $D$ будет считать, что все образцы принадлежат к оригинальной выборке.

Теперь, когда вы знаете, как в работает GAN, мы готовы реализовать свой собственный вариант нейросети, используя популярный фреймворк глубокого обучения PyTorch.

# Ваша первая генеративно-состязательная нейросеть

В качестве первого эксперимента с порождающими состязательными сетями мы реализуем пример, описанный в предыдущем разделе.

![]()

![]()

![]()

![]()

![]()

![]()

[инструкция по установке](https://pytorch.org/get-started/locally/)

In [7]:
import torch
from torch import nn

import math
import matplotlib.pyplot as plt

Здесь мы импортируем библиотеку PyTorch как `torch`. Из библиотеки мы отдельно импортируем компонент `nn` просто для того, чтобы было удобнее настраивать нейронные сети. Затем мы импортируем `math` для получения значения константы `pi` и инструмент построения графиков Matplotlib.

In [8]:
import torch

ModuleNotFoundError: No module named 'torch'